### IS Proof of concept

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Master_PJ_DRMABS/Datasource

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import re

In [ ]:
import time
from datetime import datetime
today = datetime.now()

In [ ]:
import nltk
from nltk import bigrams
import collections

In [ ]:
#!pip install pythainlp
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import dict_trie
from pythainlp.tag import pos_tag

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [ ]:
from IPython.display import clear_output

In [ ]:
# Files definition
root_path = 'Datasource'

comment_file = []
comment_file.append(root_path+"/comment_meiji.csv")
comment_file.append(root_path+"/comment_DDCF.csv")

#comment_file = []
#comment_file.append("comment_meiji.csv")
#comment_file.append("comment_DDCF.csv")

comment_nlp_file = root_path+"/comment_nlpToken.csv"
comment_cooc_nlp_file = root_path+"/comment_cooc.xlsx"

#comment_nlp_file = "comment_nlpToken.csv"
#comment_cooc_nlp_file = "comment_cooc.xlsx"

### Clean data & NLP Tokenized

In [ ]:
df_textList1 = pd.read_csv(comment_file[0],header=None)
df_textList2 = pd.read_csv(comment_file[1],header=None)

In [ ]:
df_textList1.insert(0,'Table','Meiji')
df_textList1.columns = ['Table','URLs', 'text', 'Retrived date']

In [ ]:
df_textList2.insert(0,'Table','DDFC')
df_textList2.columns = ['Table','URLs', 'text', 'Retrived date']

In [ ]:
pd.DataFrame(df_textList1.URLs.value_counts()).describe().T

In [ ]:
print('meiji comment:',df_textList1.shape[0])
print('meiji URLs ก่อนตัด:',len(df_textList1.URLs.unique()))

In [ ]:
pd.DataFrame(df_textList2.URLs.value_counts()).describe().T

In [ ]:
print('DDCF comment:',df_textList2.shape[0])
print('DDCF URLs ก่อนตัด:',len(df_textList2.URLs.unique()))

In [ ]:
# Merge df_textList1,df_textList2
df_Corpus = pd.concat([df_textList1,df_textList2],ignore_index=True)

In [ ]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

In [ ]:
missing = df_Corpus.isnull().sum()
print(missing)

In [ ]:
df_Corpus.tail()

In [ ]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','Table','URLs', 'text', 'Retrived date']

In [ ]:
df_Corpus.tail()

In [ ]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [ ]:
missing = df_Corpus.isnull().sum()
print(missing)

In [ ]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

In [ ]:
df_Corpus.insert(4,'token_text',None)
df_Corpus.tail()

In [ ]:
del df_textList1,df_textList2

### ตัดคำแบบไม่ Train เพื่อ EDA

ถ้าเคยตัดแล้วให้ข้ามได้<br>
https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [ ]:
def checkPunc(wtkn):
    newList,cnt = [],0
    for word in wtkn:
        for char in word:
            if char in set(string.punctuation):
                cnt = cnt + 1
        if cnt == 0:
            newList.append(word)
        cnt = 0
    return newList

In [ ]:
def checkStopwords(w):
    returnList = []
    stopwords = set(thai_stopwords())
    for i in w:  # comment
        if i not in stopwords:
            returnList.append(i)
    return returnList

In [ ]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i))
    return returnList

In [ ]:
# https://thainlp.org/pythainlp/docs/2.0/api/tag.html
from pythainlp.spell import correct
from pythainlp.tag import pos_tag

#https://thainlp.org/pythainlp/docs/2.0/api/transliterate.html
from pythainlp.transliterate import romanize

In [ ]:
df_process = df_Corpus.copy()

In [ ]:
corpus = []
garbage_char = ['','\n','\xa0','application','ทางกฎหมาย','html','http','https','url','com'
               'นะค่ะ','นะคะ','น่ะค่ะ','ฯ','ๆ']

In [ ]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

In [ ]:
words = set(thai_words())  # thai_words() returns frozenset
custom_tokenizer = Tokenizer(engine='newmm', custom_dict=words, keep_whitespace=False)

In [ ]:
romanize('ทดสอบ', engine="thai2rom")

In [ ]:
for i,txt in enumerate(df_Corpus['text'],0):
    print('Row:',i)
    wtkn = custom_tokenizer.word_tokenize(txt)
    wtkn = checkPunc(wtkn)
    wtkn = checkStopwords(wtkn)
    wtkn = normThai(wtkn)
    wtkn = [x for x in wtkn if x not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    df_process.iloc[i,4] = list(wtkn)
    clear_output(wait=True)

In [ ]:
df_process.head()

In [ ]:
df_process.to_csv(comment_nlp_file, index=True)

In [ ]:
del df_process,df_Corpus

### EDA Characteristic & สร้าง BoW + Term

1. Checkpoint#0 EDA รวบรวมคำจากทุก URL ทั้งหมด ทำ Unique

In [ ]:
df_EDA = pd.read_csv(comment_nlp_file,index_col=0)

In [ ]:
df_EDA.tail()

In [ ]:
tagGroup = ['ADJ','ADP','PUNCT','ADV','AUX','SYM','INTJ','CCONJ','X','NOUN','DET','PROPN','NUM','VERB','PART','PRON','SCONJ']

กำหนด TAG และ Sound

In [ ]:
row_eval = []
for i,txt in enumerate(df_EDA.token_text,0):     
    print('Row:',i+1,'of',df_EDA.shape[0])
    row_eval.append(list(eval(txt)))
    clear_output(wait=True)

In [ ]:
set_eval = set(itertools.chain.from_iterable(row_eval))
print(len(set_eval))

In [ ]:
df_EDA_withRTGS = pd.DataFrame(pos_tag(set_eval, corpus='pud'), columns=['WORD', 'TAG'])
df_EDA_withRTGS['SOUND'] = None

In [ ]:
for c,t in enumerate(df_EDA_withRTGS.WORD,0):
    print('Sound:',c+1,'of',df_EDA_withRTGS.shape[0])
    df_EDA_withRTGS.SOUND.iloc[c] = romanize(t, engine="thai2rom")
    clear_output(wait=True)

In [ ]:
df_EDA_withRTGS.to_csv(root_path+'/EDA_withRTGS.csv',index=False)

In [ ]:
del df_EDA_withRTGS

2. Checkpoint#1 วิเคราะห์รูปแบบของคำ ดูว่ามีประเภทคำแจกแจงเท่าไหร่ และมีเสียงแบบไหน<br>
export เอาข้อมูลไปดูว่า 1 การออกเสียงสามารถเขียนสะกดเป็นอะไรได้บ้าง ขั้นตอนนี้เราจะรู้ว่าเวลากรอง Term ที่เขียนผิดได้หลายแบบ<br>
จะต้องเขียนดักอย่างไรบ้าง<br>
https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp_get_started.html

In [ ]:
df_EDA2 = pd.read_csv(root_path+'/EDA_withRTGS.csv')
#df_EDA2 = pd.read_csv('EDA_withRTGS.csv')

In [ ]:
df_EDA2.tail()

In [ ]:
df_EDA2.TAG.value_counts()

In [ ]:
def checkThai(txt):
    prop = pythainlp.util.countthai(txt)
    return prop

Clean data รอบ 2 เพื่อคัดแต่คำที่เป็นภาษาไทยเท่านั้น ก่อนเริ่ม EDA เพราะว่าคำอังกฤษนั้นจัดการง่ายกว่า จึงเอาไว้ก่อน

In [ ]:
df_EDA3 = df_EDA2.copy()
df_EDA3.head()

In [ ]:
df_EDA3.shape

In [ ]:
missing = df_EDA3.isnull().sum()
print(missing)

In [ ]:
df_EDA3 = df_EDA3.dropna()

In [ ]:
missing = df_EDA3.isnull().sum()
print(missing)

In [ ]:
seqNum = list(range(1,df_EDA3.shape[0]+1))
df_EDA3.insert(0,'SEQ',seqNum)
df_EDA3.set_index('SEQ',inplace=True)

In [ ]:
indexCol = []
for i,txt in enumerate(df_EDA3.WORD,0):
    if checkThai(txt)<100:
        print(i,txt)
        indexCol.append(i+1)
    clear_output(wait=True)

In [ ]:
print(len(indexCol))
print(df_EDA3.shape)

In [ ]:
df_EDA3 = df_EDA3.drop(index=indexCol, axis=0)

จะพบว่าคำที่เป็นภาษาไทย ไม่มีอักขระอังกฤษ มีเพียง 32710 คำ ซึ่งจำนวนมากเป็นการสะกดที่ไม่มีความหมาย พิจารณาเอา thai_word() ช่วยกรองอีกรอบ แต่เอาไว้ก่อน

In [ ]:
df_EDA3.shape

เก็บไฟล์ไว้วิเคราะห์รูปแบบคำ ทำใน <b>W_ALL.xlsx</b> โดยสรุปเป็นตาราง ที่ทำแบบนี้เพื่อให้ค้นหาคำได้เร็วใน Excel
และไม่ต้องค้นหาลำบากเนื่องจากอักขระพิเศษและอังกฤษถูกตัดออกไปหมดแล้ว

In [ ]:
# เก็บไฟล์ไว้วิเคราะห์รูปแบบคำ
pd.DataFrame(df_EDA3).to_excel(root_path+'/W_ALL.xlsx',index=True)

### EDA Part
1.สรุป Term frequency ของคำ ราย Overall & URL

In [ ]:
df_EDA = pd.read_csv(comment_nlp_file)

In [ ]:
df_EDA.head()

In [ ]:
d_freq = pd.DataFrame(data=None)

row_eval,urls = [],[]
for i,txt in enumerate(df_EDA.token_text,0):     
    print('Row:',i+1,'of',df_EDA.shape[0])
    ddff = pd.DataFrame(pos_tag(list(eval(txt)), corpus='pud'),columns=['WORD','TAG'])
    ddff.insert(0,'URLs',value=df_EDA.URLs.iloc[i])
    ddff.insert(0,'commentId',value=df_EDA.commentId.iloc[i])
    d_freq = d_freq.append(ddff,ignore_index=True)
    clear_output(wait=True)

In [ ]:
d_freq.head(10)

In [ ]:
d_freq.to_csv(root_path+'/W_FRRQ.csv',index=True)

2.EDA คำรายกระทู้ การกล่าวถึงแบรนด์ในแต่ละกระทู้ และสรุป Term ที่ใช้ Train ตอนตัดคำ<br>
ทั้งนี้จะนำ thai_word() มาตัดคำให้เหลือแต่คำที่มีความหมายเท่านั้น หลังจากสรุป Freq รอบแรก และดู Freq หลังตัดอีกครั้ง

EDA Characteristic approach (Descriptive)
1. MAX คำต่อกระทู้
2. MIN คำต่อกระทู้
3. AVG คำต่อกระทู้
4. กราฟการกระจายคำต่อกระทู้
5. จำนวนกระทู้
6. พบ brand 5 ตัว ตัวละกี่กระทู้
7. กระทู้ที่มีคำเปรียบเทียบ มีกี่กระทู้
8. Term frequency

เพิ่ม commentId
8. comment มี 1 brand
9. comment มี 2 brand

In [ ]:
df_EDA_frq = pd.read_csv(root_path+'/W_FRRQ.csv',index_col=0)
df_EDA_frq.index.name = 'wordId'

In [ ]:
df_EDA_frq.tail()

In [ ]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print('จำนวนกระทู้ = ',len(df_EDA_frq.URLs.unique()))
print('จำนวนคำต่อกระทู้มากที่สุด = ',max(df_EDA_frq.URLs.value_counts()))
print('จำนวนคำต่อกระทู้น้อยที่สุด = ',min(df_EDA_frq.URLs.value_counts()))
print('จำนวนคำต่อกระทู้เฉลี่ย = ', str(round(stat.mean(df_EDA_frq.URLs.value_counts()), 2)))

In [ ]:
print('จำนวนคำต่อ comment มากที่สุด = ',max(df_EDA_frq.commentId.value_counts()))
print('จำนวนคำต่อ comment น้อยที่สุด = ',min(df_EDA_frq.commentId.value_counts()))
print('จำนวนคำต่อ comment เฉลี่ย = ', str(round(stat.mean(df_EDA_frq.commentId.value_counts()), 2)))

In [ ]:
df_plot_dist = pd.DataFrame(df_EDA_frq.URLs.value_counts())
df_plot_dist.columns = ['CNT']
sns.set(style="darkgrid")  
fig, ax = plt.subplots(figsize=(15, 5))
ax = sns.distplot(df_plot_dist.CNT, kde=True)
plt.show()

จากข้อมูลการกล่าวถึง พบว่าจำนวนนั้น imbalance กัน แต่เนื่องจากเราไม่ได้จะทำโมเดลทำนาย จึงไม่ต้อง Over-Undersample<br>
เราทำกราฟ จะต้อง normalized ตัวเลขในตาราง Cooccurence ด้วยค่า Lift ก่อน<br>

และเราพบว่าการตัดข้อความชื่อนม ฟาร์มโชคชัย มีปัญหา พบว่าจะเจอคำว่า อืมม มิลค์ (Umm) มากกว่าชื่อแบรนด์ตรงๆ

In [ ]:
print('การกล่าวถึง เมจิ :',len(df_EDA_frq[df_EDA_frq.WORD.isin(['เมจิ','meiji','cpmeiji'])].URLs.unique()),'กระทู้')
print('การกล่าวถึง โชคชัย :',len(df_EDA_frq[df_EDA_frq.WORD.isin(['โชคชัย','chokchai','ฟาร์มโชคชัย','อืมม','umm'])].URLs.unique()),'กระทู้')
print('การกล่าวถึง ดัชมิลล์ :',len(df_EDA_frq[df_EDA_frq.WORD.isin(['ดัชมิลล์','ดัชมิล','dutchmill','dutchmilk','ดัชชี่','นมดัช'])].URLs.unique()),'กระทู้')
print('การกล่าวถึง เอ็มมิลค์ :',len(df_EDA_frq[df_EDA_frq.WORD.isin(['เอ็มมิลค์','mmilk','เอ็มมิ้ลค์'])].URLs.unique()),'กระทู้')
print('การกล่าวถึง โฟร์โมสต์ :',len(df_EDA_frq[df_EDA_frq.WORD.isin(['โฟร์โมสต์','โฟร์โมส','foremost','นมโฟ'])].URLs.unique()),'กระทู้')
print('การกล่าวถึง แดรี่โฮม :',len(df_EDA_frq[df_EDA_frq.WORD.isin(['แดรี่โฮม','แดรี่','โฮม','dairy','dairy home'])].URLs.unique()),'กระทู้')

ตัดคำตาม Term ที่ต้องการ

In [ ]:
def summary_df(df):
    r_df = pd.DataFrame(df.stack(0),columns=['value'])
    r_df.index.names = ['commentId','item']
    r_df = r_df.reset_index() 
    r_df = r_df[r_df.value==1]
    return pd.DataFrame(r_df['item'].value_counts())

In [ ]:
keys = {'รสจืด','รสหวาน','ช็อคโกแลต','ช็อกโกแลต','ช๊อกโกแลต','ช๊อคโกแลต'
        ,'low fat','ไขมันต่ำ','high protein','ไฮโปรตีน','whey protein','ชงกาแฟ','รสกาแฟ','free lactose','lactose free'
        ,'นมสด','กาแฟสด','สตรอเบอรี่','สตอเบอรี่','สตรอเบอร์รี','อัลมอน','อัลมอนด์','มอลต์',
        'โชคชัย','เมจิ','meiji','ดัชมิลล์','dutchmill','แดรี่โฮม','dairy home','โฟร์โมสต์','foremost'
        ,'พาสเจอไรส์','พาสเจอไรซ์','พาสเจอร์ไรซ์','หาซื้อ','ขายดี','ขายไม่ดี','ดัชมิลล์','โฟร์โมสต์'
       ,'แพ้นม','ขวดกลาง','ขวดเล็ก','ขวดใหญ่'}
words = set(thai_words())  # thai_words() returns frozenset
for i in keys:
    words.add(i)
custom_tokenizer = Tokenizer(engine='newmm', custom_dict=words, keep_whitespace=False)

In [ ]:
for i,txt in enumerate(df_textList['text'],0):
    wtkn = custom_tokenizer.word_tokenize(txt)
    #wtkn = [w for w in wtkn if w.isalnum()]
    wtkn = [x for x in wtkn if x not in garbage_char]
    wtkn = checkPunc(wtkn)   
    wtkn = checkStopwords(wtkn)
    wtkn = [s for s in wtkn if not s.isdigit()]
    df_process.iloc[i,3] = set(wtkn)

In [ ]:
df_process.head(10)

### Grouping by word category

CONSIDERATION TERMS (ต้องลอง Review Lit ดูว่ามีอะไรบ้าง แต่ร่างคร่าวๆก่อน)
1. Brands (ยี่ห้อที่มีขายนมพาสเจอไรส์) 1.โชคชัย 2.เมจิ 3.ดัชมิลล์ 4.แดรี่โฮม 5.โฟร์โมสต์
2. Product : ใช้ Mainstream สตอเบอรี่,ช็อคโกแลต,จืด,หวาน,กาแฟ,ไขมันต่ำ,Free lactose,High protein
3. Quality
4. Price
5. Utility : ขนม กาแฟ espresso
6. Package
7. Promotion & Event เช่น เชียร์ชิม ประกวดบาริสต้า เจมส์จิ
8. Point of sales เช่น Makro 7/11 Lotus
9. Others (จับกลุ่มแล้วมาพิจารณาต่อว่ามีอะไรซ่อนอยู่อีก)

In [ ]:
colGroup = ['Brands','Product','Quality','Price','Utility','Package','Promotion_event','POS','Others']
brands = ['โชคชัย','เมจิ','meiji','ดัชมิลล์','dutchmill','แดรี่โฮม','dairy home','โฟร์โมสต์','foremost']
prds = ['สตอเบอรี่','สตรอเบอรี่','สตรอเบอร์รี','ช็อคโกแลต','ช็อกโกแลต','ช็อค','ช็อก','รสจืด','รสหวาน','free lactose','lactose free'
        ,'รสกาแฟ','ไขมันต่ำ','high protein','ไฮโปรตีน','แลคโตส','whey protein','whey','เวย์']
quality = ['บูด','เสีย','แตก','ขาด','คุณภาพ','สะอาด','ความสะอาด','จืด','หวาน','หอม','อร่อย']
price = ['ถูก','แพง']
pkg = ['ขวดกลาง','ขวดเล็ก','ขวดใหญ่','ฝา']

In [ ]:
c_grp1 = countTerm(pd.DataFrame(index=df_textList.index,columns=brands),df_process.token_text).fillna(0)
c_grp2 = countTerm(pd.DataFrame(index=df_textList.index,columns=prds),df_process.token_text).fillna(0)
c_grp3 = countTerm(pd.DataFrame(index=df_textList.index,columns=quality),df_process.token_text).fillna(0)
c_grp4 = countTerm(pd.DataFrame(index=df_textList.index,columns=price),df_process.token_text).fillna(0)
c_grp5 = countTerm(pd.DataFrame(index=df_textList.index,columns=pkg),df_process.token_text).fillna(0)

In [ ]:
c_grp1['นมเมจิ'] = c_grp1['เมจิ']+c_grp1['meiji']
c_grp1['นมดัชมิลล์'] = c_grp1['ดัชมิลล์']+c_grp1['dutchmill']
c_grp1['นมแดรี่โฮม'] = c_grp1['แดรี่โฮม']+c_grp1['dairy home']
c_grp1['นมโฟร์โมสต์'] = c_grp1['โฟร์โมสต์']+c_grp1['foremost']
c_grp1.drop(['เมจิ','meiji','ดัชมิลล์','dutchmill','แดรี่โฮม','dairy home','โฟร์โมสต์','foremost'], axis = 1, inplace=True)

In [ ]:
summary_df(c_grp1)

In [ ]:
c_grp2['รสช็อคโกแลต'] = c_grp2['ช็อคโกแลต']+c_grp2['ช็อกโกแลต']+c_grp2['ช็อก']+c_grp2['ช็อค']
c_grp2['รสสตรอเบอร์รี'] = c_grp2['สตอเบอรี่']+c_grp2['สตรอเบอรี่']+c_grp2['สตรอเบอร์รี']
c_grp2['รสไฮโปรตีน'] = c_grp2['เวย์']+c_grp2['whey protein']+c_grp2['whey']+c_grp2['ไฮโปรตีน']+c_grp2['high protein']
c_grp2['ฟรีแลคโตส'] = c_grp2['แลคโตส']+c_grp2['lactose free']+c_grp2['free lactose']
c_grp2.drop(['ช็อคโกแลต','ช็อกโกแลต','ช็อก','ช็อค','สตอเบอรี่','สตรอเบอรี่','แลคโตส','free lactose','lactose free'
             ,'high protein','สตรอเบอร์รี','เวย์','whey protein','whey','ไฮโปรตีน'], axis = 1, inplace=True)

In [ ]:
summary_df(c_grp2)

In [ ]:
summary_df(c_grp3)

In [ ]:
summary_df(c_grp4)

In [ ]:
summary_df(c_grp5)

#### สร้าง Bag of word dataframe

In [ ]:
join_df = pd.concat([c_grp1,c_grp2,c_grp3,c_grp4,c_grp5], axis=1)

In [ ]:
join_df.head(10)

### Create co-occurrence matrix จาก Bag of word

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
def cnt_Grpby(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,i].name)[df.iloc[:,j].name].value_counts())
    PAIB.index.names = ['A','B']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    _PAIB = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return _PAIB.freq.sum()

In [ ]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        if i != j:
            item_item_matrix.iloc[i,j] = cnt_Grpby(join_df,i,j)
        else:
            item_item_matrix.iloc[i,j] = 0

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_past_nlp_file, index=True)

### Create co-occurrence matrix with Lift normalization

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
def a_in_b(df,i,j):
    PAIB = pd.DataFrame(df.groupby(df.iloc[:,j].name)[df.iloc[:,i].name].value_counts())
    PAIB.index.names = ['B','A']
    PAIB.columns = ['freq']
    PAIB.reset_index(inplace=True)
    PAIB_ = PAIB[(PAIB.A>0)&(PAIB.B>0)]
    return PAIB_.freq.sum()/PAIB.freq.sum()

In [ ]:
for i in range(0,len(item_item_matrix.columns)) :
    for j in range(0,len(item_item_matrix.columns)) :
        A = join_df.iloc[:,i]
        B = join_df.iloc[:,j]
        PA = A[A!=0].count()/A.shape[0]
        PB = B[B!=0].count()/B.shape[0]
        PAB = PB*a_in_b(join_df,i,j)
        item_item_matrix.iloc[i,j] = PAB/(PA*PB)

In [ ]:
item_item_matrix.replace([np.inf, -np.inf], np.nan,inplace=True)
item_item_matrix.fillna(0,inplace=True)

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_past_nlp_file, index=True)